In [1]:
import pandas as pd
fptrain = "../../../data/sba_7a_loans_train.parquet"
fptest = "../../../data/sba_7a_loans_test.parquet"
df_train = pd.read_parquet(fptrain)
df_test = pd.read_parquet(fptest)
df = pd.concat([df_train, df_test])
df

,BorrName,BankFDICNumber,BankZip,BorrZip,NaicsCode,FranchiseCode,BusinessAge,LoanStatus,SBAGuaranteedApproval
0,Brothers Freight Management L,Not Applicable,87109,14580,484121.0,Not Applicable,Change of Ownership,PIF,3525000.0
1,EASY SPACE STORAGE LLC,58665,28403,65401,531130.0,Not Applicable,Change of Ownership,PIF,654750.0
2,H&W Endeavors Inc.,6560,43215,77493,449121.0,S0659,"Startup, Loan Funds will Open Business",PIF,150000.0
3,Imagine Technology Group LLC,4767,80202,85226,423420.0,Not Applicable,Existing or more than 2 years old,PIF,3052500.0
4,Zorn Fruherziehung LLC,33555,33880,2301,624410.0,Not Applicable,"Startup, Loan Funds will Open Business",PIF,187500.0
...,...,...,...,...,...,...,...,...,...
4588,IronPlane LLC,4255,4843,4101,454110.0,Not Applicable,Existing or more than 2 years old,PIF,79600.0
4589,ADORE HAIR & NAILS SALON LLC,17308,96813,96826,812112.0,Not Applicable,"Startup, Loan Funds will Open Business",CHGOFF,15000.0
4590,Sunberry Limited Manufacturing,6560,43215,48335,424990.0,Not Applicable,Existing or more than 2 years old,PIF,717750.0
4591,SNFood &amp; Beverage LLC,5304,54220,53023,312140.0,Not Applicable,Unanswered,PIF,175000.0


In [2]:
df["NaicsCode"] = df["NaicsCode"].apply(lambda x: x.replace(".0", ""))
df = df.drop(["BorrName"], axis=1)
dtypes_toset = {"BorrZip": 'str', "BankZip": "str", "BankFDICNumber": 'str',\
                "NaicsCode": 'str', "FranchiseCode": 'str', \
                "BusinessAge" : 'str', "LoanStatus": 'str'}

In [3]:
df = df.astype(dtypes_toset)

In [4]:
df_catvars = pd.DataFrame.from_dict({k: df[k].nunique() for k, v in dtypes_toset.items() if v in ['category', 'str']}, orient="index").reset_index()
df_catvars.columns = ["Attribute", "Unique_Values"]
df_catvars

,Attribute,Unique_Values
0,BorrZip,9057
1,BankZip,1169
2,BankFDICNumber,1116
3,NaicsCode,860
4,FranchiseCode,979
5,BusinessAge,6
6,LoanStatus,2


In [5]:
high_cardinality_attribs = ["BorrZip", "BankZip", "BankFDICNumber", "NaicsCode", "FranchiseCode"]
df.groupby("BorrZip").size()

BorrZip
10001    10
10002     1
10004     1
10005     3
10006     6
         ..
99709     1
99752     1
99801     1
99827     1
99835     1
Length: 9057, dtype: int64

## Zip Code Digit Interpretation
see the [wikipidea link](https://en.wikipedia.org/wiki/ZIP_Code#:~:text=ZIP%20Codes%20are%20numbered%20with,delivery%20addresses%20within%20that%20region.) for the zip code to see the interpretation of the digits. The first digit represents the region, the second and third digits represent the city, the fourth and fifth digits represent the delivery address location. Since using the full zip code gives us attributes with very high branching (cardinality) lead to values that have no generalizatblity (and hence cause overfitting), stopping with the third digit uses only the city information of the zip code. This gives us better generalization since zip codes that differ only in the last two digits get pooled together. So rather than using the Borrower Zip, we use the borrower city and *get a better feature*. It is the same story with *bank zip*

In [6]:
df["BorrZip"] = df["BorrZip"].apply(lambda x : str(x)[:3])
df["BankZip"] = df["BankZip"].apply(lambda x : str(x)[:3])

### Note: Reduction in Cardinality
After recoding the _BankZip_ and the _BorrZip_ the cardinality of these attributes drops a lot. Please review the original and recoded cardinalities of these attributes

In [7]:
df_catvars = pd.DataFrame.from_dict({k: df[k].nunique() for k, v in dtypes_toset.items() if v in ['category', 'str']}, orient="index").reset_index()
df_catvars.columns = ["Attribute", "Unique_Values"]
df_catvars

,Attribute,Unique_Values
0,BorrZip,814
1,BankZip,552
2,BankFDICNumber,1116
3,NaicsCode,860
4,FranchiseCode,979
5,BusinessAge,6
6,LoanStatus,2


In [8]:
dfg = df.groupby("BorrZip")["LoanStatus"].value_counts().reset_index()

In [9]:
dfg.columns

Index(['BorrZip', 'LoanStatus', 'count'], dtype='object')

## Outliers
We still have some high cardinality attributes with less than 5 instances per each unique value. The problem with these records is that:
1. We really can't test for generalization of these attributes because we don't have enough data with these values to split between training and test
2. We will overfit if we try to fit to data with this level of granularity, see for example, slide 17, in [this link](https://www.mimuw.edu.pl/~son/datamining/DM/5-decision%20tree.pdf)

So we consider these groups outliers. There are about 3.5K records in a dataset of about 23 K records with such behavior. We can analyze these as a separate group if need be and focus on the data with good generalization for the core model development.

In [10]:
high_cardinality_attribs = ["BorrZip", "BankZip", "BankFDICNumber", "NaicsCode", "FranchiseCode"]
DROP_THRESHOLD = 5
drop_these_records = {}
for attrib in high_cardinality_attribs:
    dfg = df.groupby(attrib, observed=False).size().reset_index()
    dfg.columns = [attrib, "group_size"]
    drop_these_records[attrib] = dfg[dfg.group_size < DROP_THRESHOLD][attrib].tolist()
    

In [11]:
for attrib in high_cardinality_attribs:
    df = df[~df[attrib].isin(drop_these_records[attrib])]


In [19]:
df.shape

(19516, 8)

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

## Weight of Evidence Encoding
Now that we have reduced the cardinality of the categorical attributes, we can apply a popular technique called _weight of evidence_ encoding to featurize the categorical attributes. _Weight of Evidence_ encoding is very popular in score card development for credit risk assesment. So this is a good featurization candidate for this dataset, **after** cardinality reduction. See [this article](https://ishanjainoffical.medium.com/understanding-weight-of-evidence-woe-with-python-code-cd0df0e4001e) for example for the details of calculation. This is available in the _category encoders_ package, so it is a simple enough implementation.

In [13]:
cols_to_encode = high_cardinality_attribs + ["BusinessAge"]
import category_encoders as ce
encoder = ce.WOEEncoder(cols=cols_to_encode )

In [14]:
df_train["LoanStatus"] = df_train["LoanStatus"].apply(lambda x: 0 if x == "PIF" else 1)
df_test["LoanStatus"] = df_test["LoanStatus"].apply(lambda x: 0 if x == "PIF" else 1)

In [15]:
df_train["LoanStatus"].value_counts() 

LoanStatus
0    14878
1      734
Name: count, dtype: int64

In [16]:
df_train = encoder.fit_transform(df_train, df_train["LoanStatus"])
df_test = encoder.transform(df_test)

In [17]:
fptrain = "../../../data/cleaned_sba_7a_loans_train.parquet"
fptest = "../../../data/cleaned_sba_7a_loans_test.parquet"
df_train.to_parquet(fptrain, index=False)
df_test.to_parquet(fptest, index=False)

## Result Description
A review of the training dataset post featurization reveals a set of encoded attributes that we can use for model development.

In [18]:
df_train

,BankFDICNumber,BankZip,BorrZip,NaicsCode,FranchiseCode,BusinessAge,LoanStatus,SBAGuaranteedApproval
1111,2.118754,1.801751,0.684155,1.753780,-0.02635,0.534775,1,2500.0
4252,-0.049814,-0.646709,0.458836,-0.084499,-0.02635,0.360914,0,168000.0
199,0.761117,0.455941,1.502466,0.655168,-0.02635,-0.304523,0,11250.0
6629,1.060633,-0.360753,-0.548805,0.051199,-0.02635,0.534775,0,25000.0
17554,0.143169,0.049762,0.088772,1.332567,-0.02635,-0.304523,0,127650.0
...,...,...,...,...,...,...,...,...
15595,-0.289294,-0.519817,-0.963749,-0.128951,-0.02635,-0.304523,0,4750.0
17131,0.143169,0.049762,-0.427444,0.407226,-0.02635,0.360914,0,59500.0
5248,-0.422729,0.233954,-0.707029,-0.731126,-0.02635,-0.659645,0,12500.0
2428,0.143169,0.049762,0.564196,0.433931,-0.02635,-0.304523,1,37500.0


## KMDS Logging

In [22]:

from kmds.ontology.kmds_ontology import *
from kmds.tagging.tag_types import DataRepresentationTags

In [23]:
kaw = KnowledgeApplicationWorkflow("sba_7a_loans_WOE_DR_dev", namespace=onto)
KNOWLEDGE_BASE = "sba_7a_loans_WOE_DR_dev.xml"

In [25]:

dr_obs_list = []
observation_count = 1

dr1 = DataRepresentationObservation(namespace=onto)
dr1.finding = "If we use the zip code in both borrower and bank, as is, we get many cateogories with too few records.\
This will hinder generalization. By restricting ourself to the first three digits of the zip code we can get better generalization data.\
This is done as part othis feature development experiment."
dr1.finding_sequence = observation_count
dr1.data_representation_observation_type = DataRepresentationTags.FEATURE_ENGG_OBSERVATION.value
dr_obs_list.append(dr1)


In [26]:
observation_count += 1
dr2 = DataRepresentationObservation(namespace=onto)
dr2.finding = "Even after shrinking the zip code representation for borrower and bank, we have many categories\
with less than 5 observations. We drop these observations from the data used for model development. This subset of data\
that have too few examples similar to it are deemed outliers. We can analyze these separately. A large chunk of the data\
offers good generalization, so we proceed with the analysis with the good data and set aside the outliers. These can be profiled\
and analyzed later."
dr2.finding_sequence = observation_count
dr2.data_representation_observation_type = DataRepresentationTags.FEATURE_ENGG_OBSERVATION.value
dr_obs_list.append(dr2)

In [27]:
observation_count += 1
dr3 = DataRepresentationObservation(namespace=onto)
dr3.finding = "A weight of evidence encoder is used to encode the categorical values."
dr3.finding_sequence = observation_count
dr3.data_representation_observation_type = DataRepresentationTags.DATA_TRANSFORMATION_OBSERVATION.value
dr_obs_list.append(dr3)

In [28]:
kaw.has_data_representation_observations = dr_obs_list
onto.save(KNOWLEDGE_BASE)